# Network Intrusion Detection with Deep Learning

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3" #model will be trained on GPU 0

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

C:\Users\User\Anaconda3\envs\Talos\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


## The Data

In [3]:
import numpy as np
def load_matrices(file_name):
    with open(file_name, 'rb') as f:
        A = np.load(f)
        B = np.load(f)
        C = np.load(f)
        D = np.load(f)
        E = np.load(f)
        F = np.load(f)
    return (A,B,C,D,E,F)

In [4]:
my_file =  '../CIDS-2018/10data2.npy'
train,test,y_train,y_test,ylabel_train, ylabel_test = load_matrices(my_file)

## The Model

In [5]:
from keras import backend as K
#from keras.layers import Dense, Activation, Merge, Reshape, Dropout
from keras.layers import Activation, Reshape, Input, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import InputLayer, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from keras.models import load_model
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.activations import elu
from keras.losses import binary_crossentropy

Using TensorFlow backend.


In [6]:
import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live


In [9]:
def selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * elu(x, alpha)



In [10]:
#actvation leakyrelu
LR= LeakyReLU(0.01)
LR.__name__ = 'lrelu'
#actvation prelu
#PR= PReLU(init='zero', weights=None)
#PR.__name__ = 'prelu'

In [11]:
#filepath_ae="./logtalos/taloslr_fc_weights_a.hdf"

In [12]:
from keras.callbacks import ModelCheckpoint, CSVLogger


In [26]:
def log_file_name1(first_neuron,second_neuron,batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'

    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log3LR/neuron1_{0}-neuron2_{1}-batch{2}-kernelinit_{3}-lr{4}-activ_{5}.hdf"
    log_file = s.format(first_neuron,second_neuron,batch_size,kernel_initial,lr,activation)
    return log_file

def log_file_name2(first_neuron,second_neuron,batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'
    
    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log3LR/neuron1_{0}-neuron2_{1}-batch{2}-kernelinit_{3}-lr{4}-activ_{5}.csv"
    log_file = s.format(first_neuron,second_neuron,batch_size,kernel_initial,lr,activation)
    return log_file


In [30]:
from keras.callbacks import ModelCheckpoint
filepath="./logtalos/3layer_tes_lr.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [31]:
from keras.models import Model, Sequential

In [32]:
# first we have to make sure to input data and params into the function
# add input parameters to the function
def dnn(x_train, ylabel_train, x_val, y_val, params):

    input_dim = x_train.shape[1]

    input_img = Input(shape=(input_dim,))
    
    first_neuron=params['first_neuron']
    second_neuron=params['second_neuron']
    batch_size=params['batch_size']
    kernel_initial=params['kernel_initializer']
    lr=params['lr']
    activation=params['activation']
    
    print('activation=',activation)
    
    log_file1 = log_file_name1(first_neuron,second_neuron,batch_size,kernel_initial,lr,activation)
    checkpoint2 = ModelCheckpoint(log_file1, monitor='acc', save_best_only=True, mode='max')

    
    log_file2 = log_file_name2(first_neuron,second_neuron,batch_size,kernel_initial,lr,activation)

    csv_logger2 = CSVLogger(log_file2,separator=',', append=False)
    callbacks_list2 = [checkpoint2,csv_logger2]
    
    ###AE
    
    # replace the hyperparameter inputs with references to params dictionary 
    autoencoder = Sequential()
    #encoder
    
    if activation==PReLU: 
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(second_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
                   
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(first_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
        
    else:
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(second_neuron, activation=activation, kernel_initializer=kernel_initial))
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(first_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
    
    #model = multi_gpu_model(model, gpus=1)
    autoencoder.summary()
    print(input_dim)
    
    input_img = Input(shape=(input_dim,))
    encoder_layer1 = autoencoder.layers[0]
    encoder_layer2 = autoencoder.layers[1]
    encoder = Model(input_img, encoder_layer2(encoder_layer1(input_img)))
    
    #encoder_layer1 = autoencoder.layers[0]
    #encoder_layer2 = autoencoder.layers[1]
    #encoder_layer3 = autoencoder.layers[2]
    #encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    encoder.summary()

    
    autoencoder.compile(loss='mse', optimizer='adam', metrics=['acc'])
    #model.compile(loss=params['losses'],
    #              optimizer=params['optimizer'](),
    #              metrics=['acc', fmeasure_acc])
    
   

    
    history=autoencoder.fit(x,x,
                             epochs=params['epochs'],
                             batch_size=batch_size,
                              #shuffle=True,
                              validation_split=0.2,
                              callbacks=callbacks_list,                    
                             #callbacks=[early_stopping])
                              #validation_data=(test,test),
                              verbose=1)
  

    num_classes=15
    out2 = Dense(num_classes, activation='softmax',kernel_initializer=kernel_initial)(encoder.output)
    newmodel = Model(encoder.input,out2)
    
   
    newmodel.compile(loss='categorical_crossentropy', 
                     #optimizer='adam', 
                     optimizer= Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0, amsgrad=False),
                     metrics=['acc']) 
    newmodel.summary()


    history2a=newmodel.fit(x, y,
                           epochs=params['epochs2'],
                           batch_size=batch_size,
                           validation_split=0.2,
                           callbacks=callbacks_list2,
                           #callbacks=[early_stopping],
                           #validation_data=(test,ylabel_test),
                           verbose=1)

    
    del encoder
    del autoencoder
    #del dnn_model
    
    tf.reset_default_graph()
    
    return history2a, newmodel #, encoder
    

In [33]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[70, 60, 50],
     'second_neuron':[35,30,25,20],
     'batch_size': [32,64,256],
     'epochs': [20],
     'epochs2': [30],
     'dropout': [0],
     'kernel_initializer': ['lecun_uniform','lecun_normal','he_normal','he_uniform','normal','glorot_uniform','glorot_normal'],
     'optimizer': [Adam],
     'lr':[0.1,0.01, 0.001,0.0001],#0.00001],
     'activation':[relu, elu,selu, LR, PReLU]}
     #'activation':[LR]}#, PReLU ]}

In [34]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [35]:
x=train
y=ylabel_train

In [36]:
np.shape(x)

(1619315, 80)

In [37]:
np.shape(y)

(1619315, 15)

In [ ]:

t = ta.Scan(x, y, p, dnn, experiment_no='2',grid_downsample=0.05,)



  0%|          | 0/40 [00:00<?, ?it/s]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x000001A4597FCEF0>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_7 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_8 (Dense)              (None, 50)                1300      
_________________________________________________________________
dense_9 (Dense)              (None, 80)                4080      
Total params: 10,705
Trainable params: 10,705
Non-trainable params: 0
_________________________________________________________________
80
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 80)         


  2%|▎         | 1/40 [1:41:39<66:04:41, 6099.52s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x000001A4597FCEF0>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                1050      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                4080      
Total params: 10,200
Trainable params: 10,200
Non-trainable params: 0
_________________________________________________________________
80
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)         


  5%|▌         | 2/40 [3:20:37<63:52:20, 6051.06s/it]

...

Epoch 19/30
1295452/1295452 [==============================] - 17s 13us/step - loss: 0.0642 - acc: 0.9760 - val_loss: 2.4979 - val_acc: 0.8445
Epoch 20/30
1295452/1295452 [==============================] - 17s 13us/step - loss: 0.0630 - acc: 0.9761 - val_loss: 2.4818 - val_acc: 0.8456
Epoch 21/30
1295452/1295452 [==============================] - 16s 13us/step - loss: 0.0660 - acc: 0.9761 - val_loss: 2.4805 - val_acc: 0.8458
Epoch 22/30
1295452/1295452 [==============================] - 17s 13us/step - loss: 0.0637 - acc: 0.9763 - val_loss: 2.4813 - val_acc: 0.8455
Epoch 23/30
1295452/1295452 [==============================] - 17s 13us/step - loss: 0.0621 - acc: 0.9763 - val_loss: 2.4806 - val_acc: 0.8459
Epoch 24/30
1295452/1295452 [==============================] - 17s 13us/step - loss: 0.0579 - acc: 0.9765 - val_loss: 2.4804 - val_acc: 0.8459
Epoch 25/30
1295452/1295452 [==============================] - 17s 13us/step - loss: 0.0753 - acc: 0.9754 - val_loss: 2.4845 - val_acc: 0.8456


 48%|████▊     | 19/40 [18:57:54<14:11:56, 2434.10s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x000001A4597FCEF0>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 70)                5670      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                2130      
_________________________________________________________________
dense_3 (Dense)              (None, 70)                2170      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                5680      
Total params: 15,650
Trainable params: 15,650
Non-trainable params: 0
_________________________________________________________________
80
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)         


 50%|█████     | 20/40 [20:37:42<19:26:43, 3500.19s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x000001A4597FCEF0>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1525      
_________________________________________________________________
dense_3 (Dense)              (None, 60)                1560      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                4880      
Total params: 12,825
Trainable params: 12,825
Non-trainable params: 0
_________________________________________________________________
80
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)         

In [ ]:
x.shape

In [ ]:
t.data

In [ ]:
r = ta.Reporting(t)

In [ ]:
r.best_params()